In [5]:
import time

from contextlib import contextmanager
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import lightgbm

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]

    return df, new_columns

In [12]:
train_df = pd.read_csv('../data/stack_train.csv')
test_df = pd.read_csv('../data/stack_test.csv')

sub_df = test_df[['SK_ID_CURR']].copy()
sub_df['TARGET'] = 0

folds = KFold(n_splits= 5, shuffle=True, random_state=1001)


# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

test_df['TARGET'] = 0.05*test_df['f10_dnn'] + 0.95*test_df['lgbm']#+ 0.05*test_df['ridge']

sub_df['TARGET'] = test_df['TARGET']
sub_df[['SK_ID_CURR', 'TARGET']].to_csv('../predictions/average_preds.csv', index= False)